# Bouncing Ball with Manim

Run the cells below to render a bouncing ball animation using Manim inside Jupyter.


In [7]:
%load_ext manim.utils.ipython_magic

The manim.utils.ipython_magic module is not an IPython extension.


In [8]:
%%manim -qm -v WARNING BouncingBallWithGraph
from manim import *
import numpy as np
import math

class BouncingBallWithGraph(Scene):
    def construct(self):
        floor_y = -2.7
        total_time = 10.0
        gravity = 9.8

        coefficient = ValueTracker(0.95)

        ground = Line(LEFT * 2.5, RIGHT * 0.5).shift(DOWN * 2.7 + LEFT * 3.5)
        restitution_label = VGroup(
            Text("Restitution").scale(0.45),
            DecimalNumber(num_decimal_places=2),
        ).arrange(RIGHT, buff=0.2).next_to(ground, UP, buff=0.2).shift(LEFT * 1.2)
        restitution_label[1].add_updater(lambda d: d.set_value(coefficient.get_value()))

        axes = Axes(
            x_range=[0, total_time, 2],
            y_range=[floor_y, 3.5, 1],
            x_length=5,
            y_length=4,
            axis_config={"include_ticks": False},
        ).to_corner(UR).shift(LEFT * 0.5)
        axes_labels = axes.get_axis_labels(Tex("t (s)"), Tex("height"))

        time_tracker = ValueTracker(0.0)
        velocity_tracker = ValueTracker(0.0)

        ball = Circle(radius=0.3, color=BLUE).set_fill(BLUE, opacity=0.9)
        ball.move_to(np.array([-3.5, 3.0, 0]))
        ball_radius = ball.radius

        shadow = Circle(radius=0.22, color=GRAY_E, fill_opacity=0.4)
        def update_shadow(mob):
            height = ball.get_y()
            scale = np.interp(height, [floor_y, floor_y + 3], [1.4, 0.75])
            mob.become(
                Circle(radius=0.22, color=GRAY_E, fill_opacity=0.4)
                .move_to(np.array([-3.5, floor_y + 0.02, 0]))
                .scale(scale)
            )

        shadow.add_updater(update_shadow)

        height_history = [(0.0, ball.get_y())]

        def update_ball(mob, dt):
            y = mob.get_y()
            v = velocity_tracker.get_value()
            remaining_dt = dt

            while remaining_dt > 1e-6:
                time_to_floor = None
                if v < 0:
                    a = -0.5 * gravity
                    b = v
                    c = y - (floor_y + ball_radius)
                    discriminant = b * b - 4 * a * c
                    if discriminant >= 0:
                        sqrt_disc = math.sqrt(discriminant)
                        potential_hit = (-b - sqrt_disc) / (2 * a)
                        if 0 <= potential_hit <= remaining_dt:
                            time_to_floor = potential_hit

                if time_to_floor is not None:
                    y = y + v * time_to_floor - 0.5 * gravity * time_to_floor**2
                    v = v - gravity * time_to_floor
                    y = floor_y + ball_radius
                    v = -v * coefficient.get_value()
                    remaining_dt -= time_to_floor
                else:
                    y = y + v * remaining_dt - 0.5 * gravity * remaining_dt**2
                    v = v - gravity * remaining_dt
                    remaining_dt = 0

            mob.move_to(np.array([mob.get_x(), y, 0]))
            velocity_tracker.set_value(v)
            time_tracker.increment_value(dt)
            height_history.append((time_tracker.get_value(), y))

            if abs(v) < 0.5 and abs(y - (floor_y + ball_radius)) < 1e-3:
                mob.remove_updater(update_ball)

        ball.add_updater(update_ball)

        velocity_arrow = always_redraw(
            lambda: Arrow(
                start=ball.get_center(),
                end=ball.get_center() + UP * velocity_tracker.get_value() * 0.12,
                buff=0,
                color=YELLOW,
                max_tip_length_to_length_ratio=0.15,
                stroke_width=5,
            )
        )

        graph = always_redraw(
            lambda: axes.plot_line_graph(
                [t for t, _ in height_history] if len(height_history) > 1 else [0, 0.0001],
                [y for _, y in height_history] if len(height_history) > 1 else [ball.get_y(), ball.get_y()],
                line_color=BLUE,
                add_vertex_dots=False,
            )
        )
        graph_dot = always_redraw(
            lambda: Dot(
                axes.c2p(time_tracker.get_value(), ball.get_y()),
                color=YELLOW,
                radius=0.06,
            )
        )

        self.play(Create(ground), FadeIn(ball), FadeIn(shadow), Create(axes), FadeIn(axes_labels), FadeIn(restitution_label))
        self.add(graph, graph_dot, velocity_arrow)

        self.wait(
            total_time,
            stop_condition=lambda: (
                abs(velocity_tracker.get_value()) < 0.5
                and abs(ball.get_y() - (floor_y + ball_radius)) < 1e-3
            ),
        )

        self.play(
            FadeOut(ball),
            FadeOut(shadow),
            FadeOut(ground),
            FadeOut(axes),
            FadeOut(axes_labels),
            FadeOut(graph),
            FadeOut(graph_dot),
            FadeOut(restitution_label),
            FadeOut(velocity_arrow),
        )
        self.wait(0.2)


Manim Community v0.19.0